In [1]:
from anonymizer.anonymization import Anonymizer
from anonymizer.detection import Detector, get_weights_path
from anonymizer.obfuscation import Obfuscator
from PIL import Image
import numpy as np
from aws_lambda_typing import context as context_, events, responses
import json
import requests
from typing import Dict
import io
import boto3

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
weights_path = './weights'
kernel_size, sigma, box_kernel_size = 21,2,9
obfuscator = Obfuscator(kernel_size=int(kernel_size), sigma=float(sigma), box_kernel_size=int(box_kernel_size))
detectors = {
    'face': Detector(kind='face', weights_path=get_weights_path(weights_path, kind='face')),
    'plate': Detector(kind='plate', weights_path=get_weights_path(weights_path, kind='plate'))
}
detection_thresholds = {
    'face': 0.3,
    'plate': 0.3
}

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-04-25 17:57:53.637289: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-25 17:57:53.637478: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-25 17:57:53.648915: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-25 17:57:53.648976: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-25 17:57:54.417673: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been bu

In [3]:
filename = "Oss-West_P102.jpg"
image = Image.open(f"/Users/george/Sites/anonymizer/images/{filename}").convert('RGB')

In [4]:
def start_points(size, split_size, overlap=0, region_of_interest=None):
    offset = 0

    if region_of_interest:
        size = region_of_interest[1] - region_of_interest[0]
        offset = region_of_interest[0]
    
    stride = int(split_size * (1-overlap))
    
    points = []
    counter = 0
    
    while True:
        start_point = stride * counter
        
        if start_point + split_size >= size: # last box, will potentially exceed size
            points.append((start_point+offset, size+offset))
            break
        
        points.append((start_point+offset, start_point+split_size+offset))
        counter += 1
    return points

In [5]:
split_width = 2000
split_height = 2500

X_points = start_points(image.width, split_width, overlap=0.1)
Y_points = start_points(image.height, split_height, overlap=0.1, region_of_interest=[2500, 4500])

X_points, Y_points

([(0, 2000),
  (1800, 3800),
  (3600, 5600),
  (5400, 7400),
  (7200, 9200),
  (9000, 11000),
  (10800, 12648)],
 [(2500, 4500)])

In [6]:
coords = []
for x_start, x_stop in X_points:
    for y_start, y_stop in Y_points:
        coords.append((x_start, x_stop, y_start, y_stop))

print(f'Total of {len(coords)} splits')
coords

Total of 7 splits


[(0, 2000, 2500, 4500),
 (1800, 3800, 2500, 4500),
 (3600, 5600, 2500, 4500),
 (5400, 7400, 2500, 4500),
 (7200, 9200, 2500, 4500),
 (9000, 11000, 2500, 4500),
 (10800, 12648, 2500, 4500)]

In [7]:
detections = []

from anonymizer.utils import Box

for x_start, x_stop, y_start, y_stop in coords:
    tile = image.crop((x_start, y_start, x_stop, y_stop))
    np_tile = np.array(tile)

    for kind, detector in detectors.items():
        boxes = detector.detect(np_tile, detection_threshold=detection_thresholds[kind])

        for detection in boxes:
            detections.append(Box(
                detection.x_min + x_start,
                detection.y_min + y_start,
                detection.x_max + x_start,
                detection.y_max + y_start,
                detection.score,
                detection.kind
            ))

2022-04-25 17:57:55.925949: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-25 17:57:56.627934: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key: "method" value { s: "bilinear" } } inputs { dtype: DT_FLOAT shape { dim { size: -48 } dim { size: -76 } dim { size: -77 } dim { size: 1024 } } } inputs { dtype: DT_FLOAT shape { dim { size: -23 } dim { size: 4 } } } inputs { dtype: DT_INT32 shape { dim { size: -23 } } } inputs { dtype: DT_INT32 shape { dim { size: 2 } } value { dtype: DT_INT32 tensor_shape { dim { size: 2 } } int_val: 14 } } device { type: "CPU" model: "0" num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l

In [15]:
import json
array_shape = map(lambda box: { 
            "x_min": box.x_min,
            "y_min": box.y_min,
            "x_max": box.x_max,
            "x_max": box.x_max,
            "score": box.score,
            "kind": box.kind,
         }, detections)
json.dumps(list(array_shape))

'[{"x_min": 1085.4804515838623, "y_min": 3156.8840742111206, "x_max": 1101.8235683441162, "score": 0.9589233994483948, "kind": "face"}, {"x_min": 1017.3044204711914, "y_min": 3152.9901027679443, "x_max": 1035.280466079712, "score": 0.915312647819519, "kind": "face"}, {"x_min": 1022.1309661865234, "y_min": 3152.539908885956, "x_max": 1040.8315658569336, "score": 0.9145825505256653, "kind": "face"}, {"x_min": 1442.9224729537964, "y_min": 3178.9480447769165, "x_max": 1457.6228857040405, "score": 0.8470209836959839, "kind": "face"}, {"x_min": 2786.0804677009583, "y_min": 3182.192027568817, "x_max": 2807.3037147521973, "score": 0.8171029090881348, "kind": "face"}, {"x_min": 2681.138265132904, "y_min": 3147.416889667511, "x_max": 2694.175350666046, "score": 0.701903223991394, "kind": "face"}, {"x_min": 2568.347680568695, "y_min": 3164.9415493011475, "x_max": 2576.0151028633118, "score": 0.5082163214683533, "kind": "face"}, {"x_min": 2675.8172392845154, "y_min": 3150.8538722991943, "x_max": 2

In [8]:
np_image = np.array(image)

In [9]:
obfuscated_np_image = obfuscator.obfuscate(np_image, detections)

2022-04-25 17:58:15.506145: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [10]:
obfuscated_image = Image.fromarray((obfuscated_np_image).astype(np.uint8), mode='RGB')

In [11]:
obfuscated_image.save(f"/Users/george/Sites/anonymizer/images-anonymized/{filename}")